## Creating the Chatbot

In [53]:
question = "Tips for WGU"

In [54]:
ask_me_anything(question)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 761 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 4 tokens


You asked: <b>Tips for WGU</b>

ChatBot says: <b>
Tips for WGU include: 
1. Look up the course on Reddit and check the course's Chatter to see common stumbling blocks, and be aware of when the post was made as courses can change over time. 
2. Take a break if needed. It's better to understand a concept over a two day period than to think you know it over 6 hours. 
3. When studying for exams, try to understand why a particular answer is the correct one and why the others can’t be correct. 
4. You're not beholden to do the WGU coursework. You can use your own materials as long as you can pass the WGU exams. 
5. Take a look at the course work first, and see if there were questions at the end of the chapter or question pools at the end of the course.</b>

#### Set up

In [34]:
# Importing libraries
from llama_index import SimpleDirectoryReader, GPTSimpleVectorIndex, LLMPredictor, PromptHelper
from langchain import OpenAI
from IPython.display import Markdown, display
import os

import config

import pandas as pd
import datetime as dt

#### Importing datasets

In [16]:
post_df = pd.read_csv('wgu_subreddit_posts.csv')
post_df.head()

post_id subreddit   created_utc  \
0   k9jnq0       WGU  1.607483e+09   
1   f9fmrt       WGU  1.582659e+09   
2   mb4i9q       WGU  1.616467e+09   
3   ogc63d       WGU  1.625766e+09   
4  10x1sot       WGU  1.675872e+09   

                                            selftext  \
0  Following is a list of free resources that are...   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                                            post_url  \
0  https://www.reddit.com/r/WGU/comments/k9jnq0/f...   
1                https://i.redd.it/xpsb18v1g4j41.png   
2                https://i.redd.it/ogxbs6w2woo61.jpg   
3                https://i.redd.it/49oab5hoy0a71.png   
4                https://i.redd.it/f8w825mj21ha1.jpg   

                                          post_title link_flair_text  score  \
0                    Free Resources For WGU Students             NaN    969   
1           And after 6.5 years of college... Whoop!       I'm DONE!    650   
2  The boy is refusing to sleep, so tonight he is...             NaN    623   
3        Me on the days I have a call with my mentor             NaN    603   
4        I wanted my confetti to be a bit special :)       I'm DONE!    580   

   num_comments  upvote_ratio  
0           161          1.00  
1            56          0.99  
2            28          1.00  
3            67          0.99  
4            53          0.99

In [17]:
comments_df = pd.read_csv('wgu_subreddit_comments.csv')
comments_df.head()

post_id                                            comment
0  k9jnq0  A few more:\n\nSpotify premium / Hulu / Showti...
1  k9jnq0  50% off ANY Microsoft exam voucher when you si...
2  k9jnq0  I’ll add several more to this list. Most of th...
3  k9jnq0  One of the more comprehensive resource posts I...
4  k9jnq0  also, free license for windows 10 education, w...

### Data Cleaning

In [22]:
# Converting created_utc to datetime format
post_df['created_date'] = post_df['created_utc'].apply(lambda x: dt.datetime.fromtimestamp(x))
# Adding created year column for data relevancy
post_df['created_year'] = post_df['created_date'].dt.year
post_df.head()

post_id subreddit   created_utc  \
0   k9jnq0       WGU  1.607483e+09   
1   f9fmrt       WGU  1.582659e+09   
2   mb4i9q       WGU  1.616467e+09   
3   ogc63d       WGU  1.625766e+09   
4  10x1sot       WGU  1.675872e+09   

                                            selftext  \
0  Following is a list of free resources that are...   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                                            post_url  \
0  https://www.reddit.com/r/WGU/comments/k9jnq0/f...   
1                https://i.redd.it/xpsb18v1g4j41.png   
2                https://i.redd.it/ogxbs6w2woo61.jpg   
3                https://i.redd.it/49oab5hoy0a71.png   
4                https://i.redd.it/f8w825mj21ha1.jpg   

                                          post_title link_flair_text  score  \
0                    Free Resources For WGU Students             NaN    969   
1           And after 6.5 years of college... Whoop!       I'm DONE!    650   
2  The boy is refusing to sleep, so tonight he is...             NaN    623   
3        Me on the days I have a call with my mentor             NaN    603   
4        I wanted my confetti to be a bit special :)       I'm DONE!    580   

   num_comments  upvote_ratio        created_date  created_year  
0           161          1.00 2020-12-08 21:10:20          2020  
1            56          0.99 2020-02-25 13:29:02          2020  
2            28          1.00 2021-03-22 21:32:41          2021  
3            67          0.99 2021-07-08 12:35:43          2021  
4            53          0.99 2023-02-08 09:52:36          2023

In [24]:
# Merge posts with their comments
df = post_df.merge(comments_df, on='post_id', how='left')
# Remove rows with missing comments
df = df[~df['comment'].isnull()]

In [25]:
df

post_id subreddit   created_utc  \
0       k9jnq0       WGU  1.607483e+09   
1       k9jnq0       WGU  1.607483e+09   
2       k9jnq0       WGU  1.607483e+09   
3       k9jnq0       WGU  1.607483e+09   
4       k9jnq0       WGU  1.607483e+09   
...        ...       ...           ...   
74959  111fb4p  wgu_devs  1.676311e+09   
74960  111fb4p  wgu_devs  1.676311e+09   
74961  111fb4p  wgu_devs  1.676311e+09   
74962  111fb4p  wgu_devs  1.676311e+09   
74963  111fb4p  wgu_devs  1.676311e+09   

                                                selftext  \
0      Following is a list of free resources that are...   
1      Following is a list of free resources that are...   
2      Following is a list of free resources that are...   
3      Following is a list of free resources that are...   
4      Following is a list of free resources that are...   
...                                                  ...   
74959  im really concern about this topic, not really...   
74960  im really concern about this topic, not really...   
74961  im really concern about this topic, not really...   
74962  im really concern about this topic, not really...   
74963  im really concern about this topic, not really...   

                                                post_url  \
0      https://www.reddit.com/r/WGU/comments/k9jnq0/f...   
1      https://www.reddit.com/r/WGU/comments/k9jnq0/f...   
2      https://www.reddit.com/r/WGU/comments/k9jnq0/f...   
3      https://www.reddit.com/r/WGU/comments/k9jnq0/f...   
4      https://www.reddit.com/r/WGU/comments/k9jnq0/f...   
...                                                  ...   
74959  https://www.reddit.com/r/wgu_devs/comments/111...   
74960  https://www.reddit.com/r/wgu_devs/comments/111...   
74961  https://www.reddit.com/r/wgu_devs/comments/111...   
74962  https://www.reddit.com/r/wgu_devs/comments/111...   
74963  https://www.reddit.com/r/wgu_devs/comments/111...   

                                              post_title link_flair_text  \
0                        Free Resources For WGU Students             NaN   
1                        Free Resources For WGU Students             NaN   
2                        Free Resources For WGU Students             NaN   
3                        Free Resources For WGU Students             NaN   
4                        Free Resources For WGU Students             NaN   
...                                                  ...             ...   
74959  Will the AI make our degrees worthless? thinki...             NaN   
74960  Will the AI make our degrees worthless? thinki...             NaN   
74961  Will the AI make our degrees worthless? thinki...             NaN   
74962  Will the AI make our degrees worthless? thinki...             NaN   
74963  Will the AI make our degrees worthless? thinki...             NaN   

       score  num_comments  upvote_ratio        created_date  created_year  \
0        969           161          1.00 2020-12-08 21:10:20          2020   
1        969           161          1.00 2020-12-08 21:10:20          2020   
2        969           161          1.00 2020-12-08 21:10:20          2020   
3        969           161          1.00 2020-12-08 21:10:20          2020   
4        969           161          1.00 2020-12-08 21:10:20          2020   
...      ...           ...           ...                 ...           ...   
74959      0            18          0.35 2023-02-13 11:52:53          2023   
74960      0            18          0.35 2023-02-13 11:52:53          2023   
74961      0            18          0.35 2023-02-13 11:52:53          2023   
74962      0            18          0.35 2023-02-13 11:52:53          2023   
74963      0            18          0.35 2023-02-13 11:52:53          2023   

                                                 comment  
0      A few more:\n\nSpotify premium / Hulu / Showti...  
1      50% off ANY Microsoft exam voucher when you si...  
2      I’ll add several more to t

In [28]:
# Combine all posts and comments and save to a text file
df_tmp = df[['post_title', 'selftext', 'comment']].astype(str)
agg_comments = df_tmp.groupby(['post_title', 'selftext'])['comment'].apply('. '.join).reset_index()
agg_comments

post_title  \
0                      "Easy" Courses For Acceleration?   
1                          "Please pan under the desk."   
2           "The Missing Semester of your CS Education"   
3     "We should be able to do a mid-term migration ...   
4                  "now rotate your camera 360 degrees"   
...                                                 ...   
4930      worried about D084 - Cloud Platform Solutions   
4931  ¿Best Business degree from WGU with best retur...   
4932                      “I DID IT” Mandatory Post!!🎉🎊   
4933  “I can prolly bust this out in 2 weeks” *3 mon...   
4934  🏃💨 Finished with 10 weeks to spare on my secon...   

                                               selftext  \
0     Hey everyone! I've fallen behind the last term...   
1                                                   nan   
2     There's been a few things I've thought were mi...   
3     This is what my student/program mentor emailed...   
4                                                   nan   
...                                                 ...   
4930  I am getting ready to take this one and frankl...   
4931  A little justification for your recommendation...   
4932                                                nan   
4933                                                nan   
4934                                                nan   

                                                comment  
0     Anything in the GenEds is an easy win, if you ...  
1     Lift the cat.  He could be laying on a cheat s...  
2     Nice. I continually wondered if this will be c...  
3     That's good news. My mentor hasn't responded t...  
4     Man this is the truest stuff lol. Also how oft...  
...                                                 ...  
4930  Just passed this one last night. I will try to...  
4931  Best return on investment?  No idea.  Needed a...  
4932  Congratulations!!!. Nice! Congratulations and ...  
4933  Also, if you need ANY HELP at all with this de...  
4934  You did the whole degree in one year? Any advi...  

[4935 rows x 3 columns]

In [31]:
agg_comments['combined_text'] = agg_comments.astype(str).agg('. '.join, axis=1)
all_text = ' '.join(agg_comments['combined_text'])

# Save text to txt file
f = open("textdata/all_text_reddit.txt", "w",  encoding="utf-8")
f.write(all_text)
f.close()

In [43]:
# Create a function to construct index from txt file
def construct_index(directory_path):
    # set max input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 256
    # set max chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600
    
    #define LLM (ChatGPT gpt-3.5-turbo)
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="gpt-3.5-turbo", max_tokens=num_outputs))
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)
    
    documents = SimpleDirectoryReader(directory_path).load_data()
    
    index = GPTSimpleVectorIndex(
        documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper
    )
    
    index.save_to_disk('index.json')
    
    return index

#Constructing the chatbot
def ask_me_anything(question):
    index = GPTSimpleVectorIndex.load_from_disk('index.json')
    response = index.query(question, response_mode="default")
    
    display(Markdown(f"You asked: <b>{question}</b>"))
    display(Markdown(f"ChatBot says: <b>{response.response}</b>"))

In [44]:
 # Set OpenAI key
os.environ["OPENAI_API_KEY"] = config.api_key

In [37]:
# Construct index (takes file in folder, splits into chunks, then embeds it with OpenAI's embeddings API)
construct_index('textdata')

KeyboardInterrupt: 